# `train_colab.ipynb` — Hard Hat (Helmet) Detection (YOLOv8)


**Bu notebook Colab üzerinde çalıştırılmak üzere hazırlandı.** Amacımız: kendi video/görüntü veri setini kullanarak bir **kask (helmet) / no_helmet** (kasksız) tespit modeli eğitmek, **en iyi ağırlığı (`best.pt`)** elde etmek ve bunu **ONNX** formatına dışa aktarmaktır. Notebook adım adım:

1. Ortam kurulumu (Ultralytics YOLOv8 + ONNX)
2. Veri seti yapısı ve etiket formatı (YOLO format)
3. Veri yükleme / Drive bağlama / klasör hazırlama
4. Eğitim (transfer learning — yolov8n ile başlama)
5. Değerlendirme ve görselleştirme
6. Model dışa aktarma (ONNX) ve quantizasyon (opsiyonel)
7. Eğitilmiş ağırlıkları indirme

> Not: Colab runtime'ında `Runtime > Change runtime type` menüsünden **GPU (CUDA)** seçmen eğitim hızını önemli oranda artırır.


In [ ]:
# 1) Ortam kurulumu — Bu hücreyi çalıştırın (Colab'da)
!pip install -U ultralytics onnx onnxruntime -q

# Kısa bir bilgi satırı
import ultralytics, onnx, onnxruntime, sys
print('ultralytics', ultralytics.__version__)
print('onnx', onnx.__version__)
print('onnxruntime', onnxruntime.__version__)


In [ ]:
# 2) GPU kontrolü ve device seçimi
import torch, os
print('torch', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print('GPU name:', torch.cuda.get_device_name(0))
    except:
        pass
# device: 0 for GPU, 'cpu' otherwise — ultralytics accepts both
device = 0 if torch.cuda.is_available() else 'cpu'
print('Using device =', device)


## Veri seti yapısı ve etiket formatı (YOLO format)


Eğitim için en kolay ve yaygın format **YOLO (txt)** formatıdır. Dizilim aşağıdaki gibi olmalıdır:

```
/dataset/
  /train/
    /images/
      img001.jpg
      img002.jpg
    /labels/
      img001.txt
      img002.txt
  /val/
    /images/
    /labels/
```

Her `*.txt` dosyası için satırlar şu formatta olmalı (normalleştirilmiş koordinatlar):

```
<class_id> <x_center> <y_center> <width> <height>
```

Tüm sayılar `0..1` aralığında normalize edilmiş olmalı (x_center, y_center, width, height). Örnek:

```
# örnek: class 0 = helmet, class 1 = no_helmet
0 0.5123 0.4132 0.2345 0.3210
```

**Sınıf isimlerini** bir `data.yaml` dosyasında tanımlayacağız (aşağıda oluşturacağız).

> Öneri: Annotasyon için LabelImg, CVAT veya Roboflow kullanabilirsiniz. Roboflow etiketleme, augmentasyon ve export işlemlerini kolaylaştırır. LabelImg ile YOLO formatında kayıt alabilirsiniz.


In [ ]:
# 3) Örnek klasör iskeleti oluştur (gerekirse). Gerçek verinizi buraya yükleyin veya Google Drive'a koyun.
import os
os.makedirs('/content/dataset/train/images', exist_ok=True)
os.makedirs('/content/dataset/train/labels', exist_ok=True)
os.makedirs('/content/dataset/val/images', exist_ok=True)
os.makedirs('/content/dataset/val/labels', exist_ok=True)
print('Dataset klasörleri oluşturuldu: /content/dataset/...')
print('NOT: Gerçek görüntü ve label dosyalarınızı yukarıdaki klasörlere yükleyin (Colab upload veya Drive mount).')


### Google Drive'a bağlanmak (opsiyonel)

Eğer veri setinizi Google Drive'da saklıyorsanız, aşağıdaki hücre ile Drive'ı mount edebilirsiniz. Bu, büyük veri setleriyle çalışırken pratiktir.

> `Mount` sonrası Drive içindeki yolu (ör. `/content/drive/MyDrive/datasets/myhardhat/`) `data.yaml`'da kullanabilirsiniz.


In [ ]:
# Drive'a bağlan (opsiyonel) — sadece isterseniz çalıştırın
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted: /content/drive')
except Exception as e:
    print('Drive mount çalıştırılmadı veya bu ortam Colab değil:', e)


In [ ]:
# 4) data.yaml oluştur (YOLOv8 için). Sınıf isimlerini ihtiyacınıza göre güncelleyin.
import yaml
data = {
    'train': '/content/dataset/train/images',
    'val': '/content/dataset/val/images',
    'names': ['helmet', 'no_helmet']
}
with open('data.yaml', 'w') as f:
    yaml.dump(data, f)
print('data.yaml oluşturuldu:')
print(open('data.yaml').read())


In [ ]:
# 5) Eğitim — Transfer learning ile yolov8n (küçük/nano) modeliyle başlamak hızlı sonuç verir.
# Aşağıdaki parametreleri dataset boyutuna ve Colab GPU belleğine göre ayarlayın.
from ultralytics import YOLO
import torch

device = 0 if torch.cuda.is_available() else 'cpu'

# model seçimi: 'yolov8n.pt' (nano), 'yolov8s.pt' (small) vs 'yolov8m.pt' etc.
model = YOLO('yolov8n.pt')

# Önemli parametreler:
# epochs: eğitim epoch sayısı (başlangıç 30-100 arası deneyin)
# imgsz: input boyutu (640 yaygın, 480 daha hızlı)
# batch: GPU belleğinize göre ayarlayın (ör: 16, 8, 4)
# device: 0 (GPU) veya 'cpu'

model.train(
    data='data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    workers=4,
    device=device
)

# Eğitim tamamlandığında sonuçlar runs/train/ altında olacak; en iyi ağırlık genelde runs/train/exp/weights/best.pt
print('Eğitim tamamlandı — runs/train/ klasörünü kontrol edin.')


### Eğitim ipuçları ve hiperparametreler

- **epochs**: Küçük datasetler için 30-100 arası; büyük datasetler için daha yüksek. Başlangıçta 50 iyi bir denemedir.
- **batch**: GPU belleğine göre. `Out of memory` hatası alırsanız küçültün (16 → 8 → 4).
- **imgsz**: 640 iyi bir başlangıç; hız için 480 deneyin ama doğruluk düşer.
- **lr (learning rate)**: model.train() içine `lr` argümanı verilebilir, varsayılan genelde iyi çalışır. Eğer eğitim hızlıca plateau oluyorsa lr'yi azaltın.
- **early stopping**: ultralytics otomatik olarak `patience` benzeri mekanizmalar kullanır; doğrulama mAP iyileşmezse `best.pt`'i kaydeder.
- **augmentasyon**: Roboflow veya ultralytics'in `augment` parametresiyle veri augmentasyonu yapabilirsiniz.
- **class imbalance**: Eğer `no_helmet` örnekleriniz azsa augment veya ekstra veri toplayın.


In [ ]:
# 6) Değerlendirme ve tahmin örneği
from ultralytics import YOLO

# en iyi ağırlığı yükle (eğitimden çıkan path farklı olabilir)
best_weights = 'runs/train/exp/weights/best.pt'
model = YOLO(best_weights)

# Tek bir resim üzerinde test et ve sonuçları kaydet
sample_image = '/content/dataset/val/images'  # bu klasörde en az 1 resim olmalı
# Eğer belirli bir dosya ismi biliyorsanız onu verin, örn: '/content/dataset/val/images/img001.jpg'

# Aşağıdaki predict komutu, sonuçları runs/predict/ içinde kaydeder (save=True) ve konsola özet yazdırır.
# Eğer sample_image bir klasör ise model otomatik olarak o klasördeki resimlere çalışır.
res = model.predict(source=sample_image, conf=0.25, imgsz=640, save=True)
print('Tahmin tamamlandı. Kayıt yolu örnek:', res)


In [ ]:
# 7) Doğrulama (metrics)
# ultralytics ile validation sonuçlarını almak:
model.val()  # validation run çağırır ve mAP gibi metrikleri gösterir


In [ ]:
# 8) Modeli ONNX'e dışa aktar
# Bu hücre eğitim bittikten sonra çalıştırılmalı ve best.pt yolunu model dosyanıza göre güncelleyin.
from ultralytics import YOLO
m = YOLO('runs/train/exp/weights/best.pt')
# Export ONNX (opset default). imgsz modelin beklediği giriş boyutu ile eşleşmeli (e.g., 640)
m.export(format='onnx', imgsz=640)
print('ONNX export tamamlandı — dosyayı runs/format/ altında arayın (ör: runs/format/onnx/best.onnx)')


In [ ]:
# 9) (Opsiyonel) ONNX dynamic quantization — CPU inference hızlandırabilir
try:
    from onnxruntime.quantization import quantize_dynamic, QuantType
    src = 'runs/format/onnx/best.onnx'
    dst = 'runs/format/onnx/best_quant.onnx'
    quantize_dynamic(src, dst, weight_type=QuantType.QInt8)
    print('Quantization tamamlandı:', dst)
except Exception as e:
    print('Quantization çalıştırılamadı:', e)
    print('ONNX quantization için onnxruntime-inference-tools gerekebilir veya dosya yollarını kontrol edin.')


In [ ]:
# 10) ONNXRuntime ile hızlı test (basit örnek). Model çıktısı export edilmiş ONNX formatına göre değişebilir.
import onnxruntime as ort
import numpy as np
import cv2

onnx_path = 'runs/format/onnx/best.onnx'
try:
    sess = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])
    print('ONNX loaded. Inputs:', [i.name + str(tuple(i.shape)) for i in sess.get_inputs()])
    print('Outputs:', [o.name + str(tuple(o.shape)) for o in sess.get_outputs()])
except Exception as e:
    print('ONNX yüklenemedi veya yol yanlış:', e)
    print('Eğer export farklı klasöre kaydedildiyse doğru path ile güncelleyin.')

# Not: ONNX çıktısının postprocess'i modelinize göre değişir. Backend içinde ONNX için özel bir pre/postprocess implementasyonu gerekecektir.


In [ ]:
# 11) Eğitilmiş ağırlıkları indirme (Colab için):
try:
    from google.colab import files
    files.download('runs/train/exp/weights/best.pt')
except Exception as e:
    print('files.download çalıştırılamadı (muhtemelen Colab değil). runs/train/exp/weights/best.pt yolunu manuel olarak indirin ya da Drive a kopyalayın:', e)


In [ ]:
# 12) Video üzerinde inference örneği ve snapshot kaydetme
# Bu örnek ultralytics model objesi ile video çalıştırır; her tespitte frame'i kaydedip klasöre yazar.
import cv2, os, json
from pathlib import Path

best = 'runs/train/exp/weights/best.pt'
model = YOLO(best)

video_path = '/content/sample_video.mp4'  # kendi videonuzun yolunu yazın
out_dir = '/content/video_detections'
os.makedirs(out_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print('Video açılamadı — doğru yol verdiğinizden emin olun veya önceden bir örnek video yükleyin.')
else:
    frame_no = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_no += 1
        # her n. frame üzerinde çalıştırmak isterseniz örnekleme ekleyin
        if frame_no % 5 != 0:
            continue
        # ultralytics model predict için geçici dosya kullanmak hızlı ve pratik
        tmp_path = f'/tmp/frame_{frame_no}.jpg'
        cv2.imwrite(tmp_path, frame)
        res = model.predict(source=tmp_path, conf=0.35, imgsz=640)
        # res bir list-like objedir; res[0].boxes içinde tespitler vardır
        boxes = res[0].boxes if len(res) else []
        if len(boxes):
            # snapshot kaydet
            save_path = os.path.join(out_dir, f'frame_{frame_no}.jpg')
            cv2.imwrite(save_path, frame)
            print('Detected -> saved snapshot:', save_path)
    cap.release()
    print('Video inference tamamlandı. snapshot klasörü:', out_dir)


## Son notlar, hata ayıklama ve sonraki adımlar

- Eğer `Out of memory` (OOM) hatası alırsanız: `batch` küçültün, `imgsz` küçültün (ör: 640 -> 480), veya `yolov8n` yerine daha küçük bir model kullanın.
- Etiket formatı hataları sık yapılır — label txt'lerinin isimlerinin görüntü isimleriyle bire bir uyuştuğundan emin olun (uzantı farklı olursa hata verir).
- ONNX export sonrası backend tarafında (FastAPI) `preprocess` ve `postprocess` fonksiyonlarını ONNX çıktı biçimine göre **özelleştirmeniz** gerekir.
- Colab'da eğitim sırasında `runs/train/exp` dizinini inceleyin; `results.png` görselleştirmesi ve `best.pt` bu klasörde bulunur.

---

Eğer istersen, şimdi bu notebook'u sizin veri setinizin yollarına göre **özelleştirip** çalıştırılabilir hâle getirebilirim; ayrıca eğitilmiş `best.onnx` dosyasını backend'e taşımak için gereken `export_to_onnx.py` ve `onnx` için preprocess/postprocess örnek kodunu da hazırlayabilirim.
